## Library imports

In [ ]:
##!pip install pickle5

In [1]:
# basic imports
import os
import gc
import math
import glob
import random
import itertools
import numpy as np
import pandas as pd
import pickle5 as pickle
## from tqdm.notebook import tqdm
from tqdm import tqdm

# DL library imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
## from  torch.cuda.amp import autocast, GradScaler

# metrics calculation
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold

# basic plotting library
import matplotlib.pyplot as plt
import seaborn as sns

# interactive plots
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

import warnings  
warnings.filterwarnings('ignore')

## Config parameters

In [2]:
class CFG:
    # pipeline parameters
    SEED        = 42
    TRAIN       = True
    LR_FIND     = False
    TEST        = False
    N_FOLDS     = 2 
    N_EPOCHS    = 2
    TEST_BATCH_SIZE  = 128
    TRAIN_BATCH_SIZE = 16
    NUM_WORKERS      = 4
    DATA_FRAC        = 1.0
    FOLD_TO_TRAIN    = [0,1,2,3, 4] 

    # model parameters
    MODEL_ARCH  = 'seq2seq'
    MODEL_NAME  = 'ConvSeq_v1'
    WGT_PATH    = ''
    WGT_MODEL   = ''
    PRINT_N_EPOCH = 2
    RNN_TYPE = 'LSTM'
    
    # scheduler variables
    MAX_LR    = 1e-1
    MIN_LR    = 1e-6
    SCHEDULER = 'CosineAnnealingWarmRestarts'  # ['ReduceLROnPlateau', 'None', 'OneCycleLR','CosineAnnealingLR']
    T_0       = 5      # CosineAnnealingWarmRestarts
    T_MULT    = 2      # CosineAnnealingWarmRestarts
    T_MAX     = 10     # CosineAnnealingLR

    # optimizer variables
    OPTIMIZER     = 'Adam'
    WEIGHT_DECAY  = 1e-6
    GRD_ACC_STEPS = 1
    MAX_GRD_NORM  = 1

    BUILDING_SITES_RANGE = [0,24]

In [3]:
floor_map = {"B2": -2, "B1": -1, "F1": 0, "F2": 1, "F3": 2, "F4": 3, "F5": 4, "F6": 5, "F7": 6, "F8": 7, "F9": 8,
             "1F": 0, "2F": 1, "3F": 2, "4F": 3, "5F": 4, "6F": 5, "7F": 6, "8F": 7, "9F": 8}

modelOutputDir = '.'
imuFeatures_trainPath = 'imuSeq2SeqData.pickle'
## imuFeatures_testPath  = ''
sampleCsvPath = 'sample_submission.csv'
ssubm = pd.read_csv(sampleCsvPath)
ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))

## number of time sequences to give as input to encoder
imuInputSequenceLength = 100

## max number of time sequences in decoder
wayPointMaxSequenceLength = 107

## Helper functions

In [4]:
def getBuildingName(buildingDataPath):
    return buildingDataPath.split('/')[-1].split('_')[0]

In [5]:
def find_no_of_trainable_params(model):
    total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_trainable_params

In [6]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(CFG.SEED)

In [7]:
def getSeq2SeqFeatures(imuDataPath):
    """ """
    # read features from file 
    with open(imuDataPath, 'rb') as inputFile:
        imuData = pickle.load(inputFile)        
    return imuData

In [8]:
def competitionMetric(preds, targets):
    """ The metric used in this competition """
    # position error
    meanPosPredictionError = torch.mean(torch.sqrt(
                             torch.pow((preds[:,0] - targets[:,0]), 2) + 
                             torch.pow((preds[:,1] - targets[:,1]), 2)))
    
    ## floor prediction error
    if((preds.shape[1] == 3) and (targets.shape[1] ==3)):
        meanFloorPredictionError = torch.mean(15 * torch.abs(preds[:,2] - targets[:,2]))
    else:
        meanFloorPredictionError = 0.0
    return meanPosPredictionError, meanFloorPredictionError

In [9]:
def getOptimizer(model : nn.Module):    
    if CFG.OPTIMIZER == 'Adam':
        optimizer = optim.Adam(model.parameters(), weight_decay=CFG.WEIGHT_DECAY, lr=CFG.MAX_LR)
    else:
        optimizer = optim.SGD(model.parameters(), weight_decay=CFG.WEIGHT_DECAY, lr=CFG.MAX_LR, momentum=0.9)
    return optimizer

In [10]:
def getScheduler(optimizer, dataloader_train):
    if CFG.SCHEDULER == 'OneCycleLR':
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr= CFG.MAX_LR, epochs = CFG.N_EPOCHS, 
                          steps_per_epoch = len(dataloader_train), pct_start=0.25, div_factor=10, anneal_strategy='cos')
    elif CFG.SCHEDULER == 'CosineAnnealingWarmRestarts':
        scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=CFG.T_MULT, eta_min=CFG.MIN_LR, last_epoch=-1)
    elif CFG.SCHEDULER == 'CosineAnnealingLR':
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_MAX * len(dataloader_train), eta_min=CFG.MIN_LR, last_epoch=-1)
    else:
        scheduler = None
    return scheduler

In [11]:
def getDataLoader(dataset, datasetType : str):
    if datasetType == 'train':
        batchSize = CFG.TRAIN_BATCH_SIZE
        shuffleDataset = True
    else:
        batchSize = CFG.TEST_BATCH_SIZE
        shuffleDataset = False
    
    dataLoader = DataLoader(dataset, batch_size= batchSize, shuffle=shuffleDataset,
                            num_workers=CFG.NUM_WORKERS, pin_memory=False, drop_last=False)
    return dataLoader

In [12]:
def plotTrainingResults(resultsDf, buildingName):
    # subplot to plot
    fig = make_subplots(rows=1, cols=1)
    colors = [ ('#d32f2f', '#ef5350'), ('#303f9f', '#5c6bc0'), ('#00796b', '#26a69a'),
                ('#fbc02d', '#ffeb3b'), ('#5d4037', '#8d6e63')]

    # find number of folds input df
    numberOfFolds = resultsDf['fold'].nunique()
    
    # iterate through folds and plot
    for i in range(numberOfFolds):
        data = resultsDf[resultsDf['fold'] == i]
        fig.add_trace(go.Scatter(x=data['epoch'].values, y=data['trainPosLoss'].values,
                                mode='lines', visible='legendonly' if i > 0 else True,
                                line=dict(color=colors[i][0], width=2),
                                name='{}-trainPossLoss-Fold{}'.format(buildingName, i)),row=1, col=1)

        fig.add_trace(go.Scatter(x=data['epoch'], y=data['valPosLoss'].values,
                                 mode='lines+markers', visible='legendonly' if i > 0 else True,
                                 line=dict(color=colors[i][1], width=2),
                                 name='{}-valPosLoss-Fold{}'.format(buildingName,i)),row=1, col=1)
    fig.show()

## Dataset class

In [13]:
class imuFeaturesDataset_train(Dataset):
    def __init__(self, imuData):
        self.imuData = imuData 
        
    def __getitem__(self, index):
        ## output shape is (imuInputSequenceLength, 7)
        ## where 7 indicates numbre of features - [ts, lin_ax, lin_ay, gz, roll, pitch, yaw]
        ## imuInputSequenceLength is max input sequence length
        encoderData  = self.imuData['encoderData'][index].transpose()
        encoderData  = torch.from_numpy(encoderData)

        ## output shape is (wayPointMaxSequenceLength, 3)
        ## where 3 indicates numbere of targets - [x,y,floor]
        decoderData  = self.imuData['decoderData'][index]  ## .transpose()
        decoderData  = torch.from_numpy(decoderData)

        ## output shape is (1, wayPointMaxSequenceLength)
        ## where 1 indicates inference Ts - decoder input
        inferenceTs  = self.imuData['inferenceTsList'][index]
        inferenceTs  = torch.from_numpy(inferenceTs)
        
        ## path name
        pathName = self.imuData['path'][index]
        ## number of waypoints
        numWayPoints = self.imuData['numWayPoints'][index].astype(np.int16)

        return encoderData, decoderData, inferenceTs, pathName, numWayPoints
     
    def __len__ (self):
        return len(self.imuData)

## Convolution seq2seq Model classes - Encoder, Decoder and Seq2Seq

In [14]:
class Encoder(nn.Module):
    def __init__(self, inputSize, embSize, hidSize, nLayers, \
                 kernelSize, dropout, device, maxSeqLen = 100):
        
        super().__init__()
        assert kernelSize % 2 == 1, "Kernel size must be odd!"
        self.scale = torch.sqrt(torch.FloatTensor([0.5])).to(device)
        
        ## store variables
        self.encFeatSize = inputSize
        self.encEmbSize  = embSize
        self.encHidSize  = hidSize
        self.nLayers     = nLayers
        self.kernelSize  = kernelSize
        self.device      = device
        self.encSeqLen   = maxSeqLen
        
        ## encoder layers
        self.tokEmbedding = nn.Linear(self.encFeatSize, self.encEmbSize)
        self.posEmbedding = nn.Embedding(self.encSeqLen, self.encEmbSize)
        self.emb2hid = nn.Linear(self.encEmbSize, self.encHidSize)
        self.hid2emb = nn.Linear(self.encHidSize, self.encEmbSize)
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = self.encHidSize, out_channels = 2 * self.encHidSize, \
                                    kernel_size = self.kernelSize,  padding = (self.kernelSize - 1) // 2)
                                    for _ in range(self.nLayers)])
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        ## src = [batchSize, encSeqLen, encFeatSize]
        batchSize = src.shape[0]
        
        ## 1. create position tensor
        ## pos = [0, 1, 2, 3, ..., encSeqLen - 1], repeated for batchSize, along rows
        ## pos = [batchSize, encSeqLen]
        pos = torch.arange(0, self.encSeqLen).unsqueeze(0).repeat(batchSize, 1).to(self.device)
              
        ## 2. embed tokens and positions
        ## tokEmbedded = posEmbedded = [batchSize, encSeqLen, encEmbSize]        
        tokEmbedded = self.tokEmbedding(src)
        posEmbedded = self.posEmbedding(pos)
        
        ## 3. combine embeddings by elementwise summing
        ## embedded = [batchSize, encSeqLen, encEmbSize]
        embedded = self.dropout(tokEmbedded + posEmbedded)
        
        ## 4. pass embedded through linear layer to convert from emb dim to hid dim
        ## convInput = [batchSize, encSeqLen, encHidSize]
        convInput = self.emb2hid(embedded)
        
        ## 5. permute for convolutional layer
        ## convInput = [batchSize, encHidSize, encSeqLen]
        convInput = convInput.permute(0, 2, 1) 
        
        ## 6. begin convolutional blocks..., repeate for nLayers
        for i, conv in enumerate(self.convs):
        
            ## 7. pass through convolutional layer
            ## conved = [batchSize, 2*encHidSize, encSeqLen]
            conved = conv(self.dropout(convInput))

            ## 8. pass through GLU activation function
            ## conved = [batchSize, encHidSize, encSeqLen]
            conved = F.glu(conved, dim = 1)

            ## 9. apply residual connection
            ## conved = [batchSize, encHidSize, encSeqLen]
            conved = (conved + convInput) * self.scale

            ## 10. set convInput to conved for next loop iteration
            convInput = conved
        
        ## ...end convolutional blocks
        
        ## 11. permute and convert back to emb dim
        ## conved = [batchSize, encSeqLen, encEmbSize]
        conved = self.hid2emb(conved.permute(0, 2, 1))
        
        ## 12. elementwise sum output (conved) and input (embedded) to be used for attention
        ## combined = [batchSize, encSeqLen, encEmbSize]
        combined = (conved + embedded) * self.scale
        return conved, combined

In [24]:
class Decoder(nn.Module):
    def __init__(self, inputSize, embSize, hidSize, outputSize, nLayers, \
                 kernelSize, dropout, fillerValue, device, maxSeqLen = 107):
        
        super().__init__()
        self.scale = torch.sqrt(torch.FloatTensor([0.5])).to(device)
        
        ## store vairables
        self.decFeatSize = inputSize
        self.decEmbSize  = embSize
        self.decHidSize  = hidSize
        self.decOutSize  = outputSize        
        self.nLayers     = nLayers
        self.kernelSize  = kernelSize
        self.fillerValue = fillerValue
        self.device      = device        
        self.decSeqLen   = maxSeqLen
        
        
        ## decoder layers
        self.tokEmbedding = nn.Linear(self.decFeatSize, self.decEmbSize)
        self.posEmbedding = nn.Embedding(self.decSeqLen, self.decEmbSize)
        self.emb2hid = nn.Linear(self.decEmbSize, self.decHidSize)
        self.hid2emb = nn.Linear(self.decHidSize, self.decEmbSize)
        
        ## attention specific layers
        self.attnHid2emb = nn.Linear(self.decHidSize, self.decEmbSize)
        self.attnEmb2hid = nn.Linear(self.decEmbSize, self.decHidSize)
        self.fcOut = nn.Linear(self.decEmbSize, self.decOutSize)

        ## convolution block layers
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = self.decHidSize, out_channels = 2 * self.decHidSize, \
                                    kernel_size = self.kernelSize) for _ in range(self.nLayers)])
        self.dropout = nn.Dropout(dropout)

      
    def calculateAttention(self, embedded, conved, encoderConved, encoderCombined):
        
        ## embedded = [batchSize, decSeqLen, decEmbSize]
        ## conved = [batchSize, decHidSize, decSeqLen]
        ## encoderConved = encoderCombined = [batchSize, encSeqLen, decEmbSize]
        
        ## a. permute and convert back to emb dim
        ## convedEmb = [batchSize, decSeqLen, decEmbSize]
        convedEmb = self.attnHid2emb(conved.permute(0, 2, 1))
        
        ## b. residual connection
        ## combined = [batchSize, decSeqLen, decEmbSize]
        combined = (convedEmb + embedded) * self.scale
        
        ## c. Energy matrix calculation
        ## energy = [batchSize, decSeqLen, encSeqLen]
        energy = torch.matmul(combined, encoderConved.permute(0, 2, 1))
        
        ## d. attention is softmax of energy
        ## attention = [batchSize, decSeqLen, encSeqLen]
        attention = F.softmax(energy, dim=2)
        
        ## e. attention over encoded states 
        ## attendedEncoding = [batchSize, decSeqLen, decEmbSize]
        attendedEncoding = torch.matmul(attention, encoderCombined)
        
        ## f. convert from emb dim -> hid dim
        ## attendedEncoding = [batchSize, decSeqLen, decHidSize]
        attendedEncoding = self.attnEmb2hid(attendedEncoding)
        
        ## g. apply residual connection with decoder token 
        ## attendedCombined = [batch size, decHidSize, decSeqLen]
        attendedCombined = (conved + attendedEncoding.permute(0, 2, 1)) * self.scale
        return attention, attendedCombined
        
    
    def forward(self, trg, encoderConved, encoderCombined):
        ## trg = [batchSize, decSeqLen]
        ## encoderConved = encoderCombined = [batchSize, encSeqLen, decEmbSize]
        batchSize = trg.shape[0]
            
        ## 1. create position tensor
        ## pos = [0, 1, 2, 3, ..., decSeqLen - 1], repeated for batchSize, along rows
        ## pos = [batchSize, decSeqLen]        
        pos = torch.arange(0, self.decSeqLen).unsqueeze(0).repeat(batchSize, 1).to(self.device)
        
        ## 2. embed tokens and positions
        ## tokEmbedded = posEmbedded = [batchSize, decSeqLen, decEmbSize] 
        trg = trg.unsqueeze(2)
        tokEmbedded = self.tokEmbedding(trg)
        posEmbedded = self.posEmbedding(pos)
        
        ## 3. combine embeddings by elementwise summing
        ## embedded = [batchSize, decSeqLen, decEmbSize]
        embedded = self.dropout(tokEmbedded + posEmbedded)
        
        ## 4. pass embedded through linear layer to convert from emb dim to hid dim
        ## convInput = [batchSize, decSeqLen, decHidSize]
        convInput = self.emb2hid(embedded)
        
        ## 5. permute for convolutional layer
        ## convInput = [batchSize, decHidSize, decSeqLen]
        convInput = convInput.permute(0, 2, 1)         
        padding = torch.zeros(batchSize, self.decHidSize, self.kernelSize - 1).fill_(\
                       self.fillerValue).to(self.device)
        
        ## 6. begin of convolution blocks 
        for i, conv in enumerate(self.convs):
        
            ## 7. apply dropout
            convInput = self.dropout(convInput)
        
            ## 8. need to pad so decoder can't "cheat"
            ## paddedConvInput = [batchSize, decHidSize, decSeqLen + kernelSize - 1]
            paddedConvInput = torch.cat((padding, convInput), dim = 2)
        
            ## 9. pass through convolutional layer
            ## conved = [batchSize, 2 * decHidSize, decSeqLen]
            conved = conv(paddedConvInput)

            ## 10. pass through GLU activation function
            ## conved = [batchSize, decHidSize, decSeqLen]
            conved = F.glu(conved, dim = 1)

            ## 11. calculate attention
            ## attention = [batchSize, decSeqLen, encSeqLen]
            ## conved = [batch size, decHidSize, decSeqLen]
            attention, conved = self.calculateAttention(embedded, conved, 
                                encoderConved, encoderCombined)
            
            ## 12. apply residual connection
            ## conved = [batchSize, decHidSize, decSeqLen]
            conved = (conved + convInput) * self.scale
                        
            ## 13. set convInput to conved for next loop iteration
            convInput = conved
            
        ## 14. convert to decEmbSize
        #conved = [batchSize, decSeqLen, decEmbSize]
        conved = self.hid2emb(conved.permute(0, 2, 1))
         
        ## 15. linear layer to output dimension
        ## output = [batchSize, decSeqLen, decOutSize]
        output = self.fcOut(self.dropout(conved))
        return output, attention

In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, src, trg):
        ## src = [batchSize, encSeqLen, encFeatSize]
        ## trg = [batchSize, decSeqLen]
        
        ## 1. calculate z^u (encoderConved) and (z^u + e) (encoderCombined)
        ## encoderConved is output from final encoder conv. block
        ## encoderCombined = encoderConved + src embedding + positional embeddings (all elementwise)
        ## encoderConved = encoderCombined = [batchSize, encSeqLen, decEmbSize]
        encoderConved, encoderCombined = self.encoder(src)
                    
        ## 2. calculate predictions of next words
        ## output is a batch of predictions for each input in the decoder
        ## attention a batch of attention scores across the encoder sequence for 
        ## each timestamp in the decoder
        ## output = [batchSize, decSeqLen, decOutSize]
        ## attention = [batchSize, decSeqLen, encSeqLen]
        output, attention = self.decoder(trg, encoderConved, encoderCombined)        
        return output, attention

In [17]:
## Device as cpu or tpu
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:0


In [18]:
data = getSeq2SeqFeatures(imuFeatures_trainPath)
print(data.shape)
## data.head(1)

(10877, 6)


In [19]:
trainDataset = imuFeaturesDataset_train(data)
trainDataLoader = getDataLoader(trainDataset, datasetType='train')
batch = next(iter(trainDataLoader))
encoderInput, decoderOutput, decoderInput, pathName, _ = batch
encoderInput  = encoderInput.to(device)
decoderOutput = decoderOutput.to(device)
decoderInput  = decoderInput.to(device)

In [20]:
encoderInput.shape, decoderOutput.shape, decoderInput.shape, len(pathName)

(torch.Size([16, 100, 7]), torch.Size([16, 107, 3]), torch.Size([16, 107]), 16)

In [21]:
INPUT_DIM = 7
DECODER_INPUT_DIM = 1
OUTPUT_DIM = 3
ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
HID_DIM = 64
N_LAYERS = 5
KERNEL_SIZE = 3
FILLER_VALUE = 0.0
ENC_DROPOUT = 0.4
DEC_DROPOUT = 0.4

In [25]:
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, KERNEL_SIZE, \
              ENC_DROPOUT, device, imuInputSequenceLength).double()
dec = Decoder(DECODER_INPUT_DIM, DEC_EMB_DIM, HID_DIM, OUTPUT_DIM, \
              N_LAYERS, KERNEL_SIZE, DEC_DROPOUT, FILLER_VALUE, device, wayPointMaxSequenceLength).double()
model = Seq2Seq(enc, dec)
model.to(device);

In [26]:
preds, _ = model(encoderInput, decoderInput)

In [27]:
preds.shape

torch.Size([16, 107, 3])

In [28]:
decoderOutput.shape

torch.Size([16, 107, 3])

## Lr range finder

In [ ]:
def plot_lr_finder_results(lr_finder): 
    # Create subplot grid
    fig = make_subplots(rows=1, cols=2)
    # layout ={'title': 'Lr_finder_result'}
    
    # Create a line (trace) for the lr vs loss, gradient of loss
    trace0 = go.Scatter(x=lr_finder['log_lr'], y=lr_finder['smooth_loss'],name='log_lr vs smooth_loss')
    trace1 = go.Scatter(x=lr_finder['log_lr'], y=lr_finder['grad_loss'],name='log_lr vs loss gradient')

    # Add subplot trace & assign to each grid
    fig.add_trace(trace0, row=1, col=1);
    fig.add_trace(trace1, row=1, col=2);
    iplot(fig, show_link=False)
    #fig.write_html(CFG.MODEL_NAME + '_lr_find.html');

In [ ]:
def find_lr(model, optimizer, data_loader, init_value = 1e-8, final_value=100.0, beta = 0.98, num_batches = 200):
    assert(num_batches > 0)
    mult = (final_value / init_value) ** (1/num_batches)
    lr = init_value
    optimizer.param_groups[0]['lr'] = lr
    batch_num = 0
    avg_loss = 0.0
    best_loss = 0.0
    smooth_losses = []
    raw_losses = []
    log_lrs = []
    dataloader_it = iter(data_loader)
    progress_bar = tqdm(range(num_batches))                
        
    for idx in progress_bar:
        batch_num += 1
        try:
            _, inputs, targets = next(dataloader_it)
            #print(images.shape)
        except:
            dataloader_it = iter(data_loader)
            _, inputs, targets = next(dataloader_it)

        # Move input and label tensors to the default device
        inputs = inputs.to(device)
        targets = targets.to(device)

        # handle exception in criterion
        try:
            # Forward pass
            y_preds = model(inputs)
            posLoss, floorLoss = criterion(y_preds, targets)
            loss = posLoss + floorLoss
        except:
            if len(smooth_losses) > 1:
                grad_loss = np.gradient(smooth_losses)
            else:
                grad_loss = 0.0
            lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                                 'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
            return lr_finder_results 
                    
        #Compute the smoothed loss
        avg_loss = beta * avg_loss + (1-beta) *loss.item()
        smoothed_loss = avg_loss / (1 - beta**batch_num)
        
        #Stop if the loss is exploding
        if batch_num > 1 and smoothed_loss > 50 * best_loss:
            if len(smooth_losses) > 1:
                grad_loss = np.gradient(smooth_losses)
            else:
                grad_loss = 0.0
            lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                                 'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
            return lr_finder_results
        
        #Record the best loss
        if smoothed_loss < best_loss or batch_num==1:
            best_loss = smoothed_loss
        
        #Store the values
        raw_losses.append(loss.item())
        smooth_losses.append(smoothed_loss)
        log_lrs.append(math.log10(lr))
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print info
        progress_bar.set_description(f"loss:{loss.item()},smoothLoss: {smoothed_loss},lr:{lr}")

        #Update the lr for the next step
        lr *= mult
        optimizer.param_groups[0]['lr'] = lr
    
    grad_loss = np.gradient(smooth_losses)
    lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                         'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
    return lr_finder_results

In [ ]:
if CFG.LR_FIND == True:
    # create dataset instance
    tempTs, tempX, tempY,_ = getBuildingData(buildingCsvPath=buildingsList[0])
    tempX = stdScaler.fit_transform(tempX)
    tempTrainDataset = wiFiFeaturesDataset(tempTs, tempX, tempY)
    tempTrainDataloader = DataLoader(tempTrainDataset, batch_size= CFG.TRAIN_BATCH_SIZE, shuffle=True,
                          num_workers=CFG.NUM_WORKERS, pin_memory=False, drop_last=False)
    
    # create model instance   
    model = wiFiFeaturesMLPModel(n_input=tempX.shape[1], n_output=3)
    model.to(device);
    
    # optimizer function, lr schedulers and loss function
    optimizer = getOptimizer(model)
    lrFinderResults = find_lr(model, optimizer, tempTrainDataloader)
    plot_lr_finder_results(lrFinderResults)
    del tempX, tempY, tempTrainDataset, tempTrainDataloader, model, optimizer

## Train & Validate helper functions

In [ ]:
def validateModel(model, validationDataloader):
    # placeholders to store output
    val_ts = []
    val_preds = []
    val_targets = []
    val_groups = []

    # set model to Validate mode
    model.eval()
    dataLoaderIterator = iter(validationDataloader)

    for idx in range(len(validationDataloader)):
        try:
            ts, inputs, targets, valGroups = next(dataLoaderIterator)
        except StopIteration:
            dataLoaderIterator = iter(validationDataloader)
            ts, inputs, targets, valGroups = next(dataLoaderIterator)

        inputs = inputs.to(device)
        targets = targets.to(device) 

        # forward prediction
        with torch.no_grad():    
            y_preds = model(inputs)

        # store predictions and targets to compute metrics later
        val_ts.append(ts)
        val_preds.append(y_preds)
        val_targets.append(targets)
        val_groups.append(valGroups)

    # concatenate to get as 1 2d array and find total loss  
    val_preds = torch.cat(val_preds, 0)
    val_targets = torch.cat(val_targets, 0)
    valPosLoss, valFloorLoss = criterion(val_preds, val_targets)
    valScore = valPosLoss + valFloorLoss

    # np array concatenation
    val_ts = np.concatenate(val_ts, axis=0)
    val_groups = np.concatenate(val_groups, axis=0)
    
    # store results
    validationResults = {'valPosLoss': valPosLoss.item() , 'valFloorLoss': valFloorLoss.item(),\
                         'val_ts': val_ts, 'val_groups': val_groups,
                         'val_preds'  :val_preds.cpu().data.numpy(), 
                         'val_targets':val_targets.cpu().data.numpy(),
                         }
    return validationResults

In [ ]:
def trainValidateOneFold(buildingName, i_fold, model, optimizer, scheduler, dataloader_train, dataloader_valid):
    trainFoldResults = []
    bestValScore = np.inf
    bestEpoch = 0

    ## for epoch in tqdm(range(CFG.N_EPOCHS)):
    for epoch in range(CFG.N_EPOCHS):
        #print('Epoch {}/{}'.format(epoch + 1, CFG.N_EPOCHS))
        model.train()
        trainPosLoss = 0.0
        trainFloorLoss = 0.0

        # training iterator
        tr_iterator = iter(dataloader_train)

        for idx in range(len(dataloader_train)):
            try:
                _, inputs, targets, _ = next(tr_iterator)
            except StopIteration:
                tr_iterator = iter(dataloader_train)
                _, inputs, targets, _ = next(tr_iterator)

            inputs = inputs.to(device)
            targets = targets.to(device)  

            # builtin package to handle automatic mixed precision
            ## with autocast():
            # Forward pass
            y_preds = model(inputs)   
            posLoss, floorLoss = criterion(y_preds, targets)
            loss = posLoss + floorLoss

            # Backward pass
            ## scaler.scale(loss).backward()        
            ## scaler.step(optimizer)
            ## scaler.update()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad() 

            # log the necessary losses
            trainPosLoss   += posLoss.item()
            trainFloorLoss += floorLoss.item()

            if scheduler is not None: 
                if CFG.SCHEDULER == 'CosineAnnealingWarmRestarts':
                    scheduler.step(epoch + idx / len(dataloader_train)) 
                # onecyle lr scheduler / CosineAnnealingLR scheduler
                else:
                    scheduler.step()
                    
        # Validate
        foldValidationResults = validateModel(model, dataloader_valid)
         
        # store results
        trainFoldResults.append({ 'fold': i_fold, 'epoch': epoch, 
                                  'trainPosLoss': trainPosLoss / len(dataloader_train), 
                                  'trainFloorLoss': trainFloorLoss / len(dataloader_train), 
                                  'valPosLoss'  : foldValidationResults['valPosLoss'] , 
                                  'valFloorLoss': foldValidationResults['valFloorLoss']})
        
        valScore = foldValidationResults['valPosLoss'] # + foldVal['valFloorLoss']
        ## print(f'fold = {i_fold}, epoch = {epoch}, valscore = {valScore}')
        # save best models        
        if(valScore < bestValScore):
            # reset variables
            bestValScore = valScore
            bestEpoch = epoch

            # save model weights
            torch.save({'model': model.state_dict(), 'val_ts' : foldValidationResults['val_ts'], 
                        'val_preds':foldValidationResults['val_preds'], 
                        'val_targets':foldValidationResults['val_targets'],
                        'val_groups' : foldValidationResults['val_groups']}, 
                        f"{modelOutputDir}/{buildingName}_{CFG.MODEL_NAME}_fold{i_fold}_best.pth")

    print(f"For Fold {i_fold}, Best position validation score of {bestValScore} was got at epoch {bestEpoch}") 
    return trainFoldResults

In [ ]:
def trainValidateOneBuilding(buildingDataPath, modelToFit):
    # placeholder to store results
    buildingTrainResults = []
    
    buildingName = getBuildingName(buildingDataPath)
    print(f"Processing data for building - {buildingName}")
    timestamps, X, y, groups = getBuildingData(buildingDataPath)
    print(f"Building Data shapes : {timestamps.shape, X.shape, y.shape, groups.shape}")

    for i_fold, (trainIndex, validIndex) in enumerate(folds.split(X=X, y=y[:,0],groups=groups)):
        if i_fold in CFG.FOLD_TO_TRAIN:
            ## print("Fold {}/{}".format(i_fold + 1, CFG.N_FOLDS))
            
            # splitting into train and validataion sets
            trainTimeStamps, X_train, y_train, trainGroups = timestamps[trainIndex], X[trainIndex], y[trainIndex], groups[trainIndex]
            validTimeStamps, X_valid, y_valid, validGroups = timestamps[validIndex], X[validIndex], y[validIndex], groups[validIndex] 
                        
            # create torch Datasets and Dataloader for each fold's train and validation data
            dataset_train = wiFiFeaturesDataset(trainTimeStamps, X_train, y_train, trainGroups)
            dataset_valid = wiFiFeaturesDataset(validTimeStamps, X_valid, y_valid, validGroups)            
            dataloader_train = getDataLoader(dataset_train, datasetType= 'train')
            dataloader_valid = getDataLoader(dataset_valid, datasetType= 'valid')
            
            # supervised model instance and move to compute device
            model = modelToFit(X.shape[1], 3, CFG.FILTERLIST, CFG.STRIDELIST)
            model.to(device);
            ## print(f"there are {find_no_of_trainable_params(model)} params in model")

            # optimizer function, lr schedulers and loss function
            optimizer = getOptimizer(model)
            scheduler = getScheduler(optimizer, dataloader_train)
            # print(f"optimizer={optimizer}, scheduler={scheduler}, loss_fn={criterion}")

            # train and validate single fold
            foldResults = trainValidateOneFold(buildingName, i_fold, model, optimizer, scheduler,dataloader_train, dataloader_valid)
            buildingTrainResults = buildingTrainResults + foldResults
            
            ## del trainTimeStamps, X_train, y_train, trainGroups
            ## del validTimeStamps, X_valid, y_valid, validGroups
            del dataloader_train, dataloader_valid, model, optimizer, scheduler
            gc.collect()
    
    ## del timestamps, X, y, groups
    ## gc.collect()
    
    buildingTrainResults = pd.DataFrame(buildingTrainResults)
    buildingTrainResults['valTotalLoss'] = buildingTrainResults['valPosLoss'] + buildingTrainResults['valFloorLoss']
    buildingTrainResults['trainTotalLoss'] = buildingTrainResults['trainPosLoss'] + buildingTrainResults['trainFloorLoss']
    return buildingTrainResults

In [ ]:
def getFoldBestResultsDf(trainResults):
    bestResults = []
    numFolds = trainResults['fold'].nunique()
    
    for fold in range(numFolds):
        if fold in CFG.FOLD_TO_TRAIN:
            foldDf = trainResults[trainResults['fold']== fold]
            bestResults.append(foldDf.iloc[np.argmin(foldDf['valTotalLoss'].values),:])
    
    bestResults =pd.DataFrame(bestResults)
    valPosLossBest = bestResults['valPosLoss'].values
    print(f"Best valPosLoss for all folds = {valPosLossBest}")
    print(f"Mean, std ={valPosLossBest.mean()}, {valPosLossBest.std()}")
    return bestResults

## Generate OOF function

In [ ]:
def generateOOF(modelSaveDir:str, buildingName:str, modelName:str):
    oof_ts = []
    oof_preds = []
    oof_targets = []
    oof_groups = []
    oof_folds = []

    modelPaths = sorted(glob.glob(f"{modelSaveDir}/{buildingName}_{modelName}_fold*.pth"))

    for fold in range(len(modelPaths)):
        # load building-model-fold checkpoint
        checkPoint = torch.load(modelPaths[fold], map_location=torch.device(device))
        numRows = len(checkPoint['val_ts'])

        oof_ts.append(checkPoint['val_ts'])
        oof_preds.append(checkPoint['val_preds'])
        oof_targets.append(checkPoint['val_targets'])
        oof_groups.append(checkPoint['val_groups'])
        oof_folds.append([fold] * numRows)
    
    oof_ts = np.concatenate(oof_ts,axis=0)
    oof_preds = np.concatenate(oof_preds,axis=0)
    oof_targets = np.concatenate(oof_targets,axis=0)
    oof_groups = np.concatenate(oof_groups,axis=0)
    oof_folds = np.concatenate(oof_folds,axis=0)
    
    #print(oof_ts.shape, oof_preds.shape, oof_targets.shape, oof_groups.shape, oof_folds.shape)
    oof_df = pd.DataFrame({'timestamp' : oof_ts, 'x_preds': oof_preds[:,0], 'y_preds': oof_preds[:,1],
                       'floor_preds': oof_preds[:,2], 'x_tgt': oof_targets[:,0], 'y_tgt': oof_targets[:,1],
                       'floor_tgt': oof_targets[:,2], 'path' : oof_groups, 'fold' : oof_folds
                      })
    print(f"OOF prediction for {buildingName} site generated")
    return oof_df

## Test set prediction function

In [ ]:
def generateWiFiSubmission(modelToFit, modelSaveDir:str, buildingName:str, modelName:str):
    modelPaths = sorted(glob.glob(f"{modelSaveDir}/{buildingName}_{modelName}_fold*.pth"))
    buildingTestData = f"{wifiFeaturesDir_test}/{buildingName}_test.pickle"
    with open(buildingTestData, 'rb') as inputFile:
        testData = pickle.load(inputFile)    
    
    test_ts = []
    test_fold = []
    test_preds = []
    test_groups = []

    for fold in range(CFG.N_FOLDS):
        ## print(f"Fold {fold} processing")
        #print(f"Before stdscaler : testX mean = {testX.mean()}, testData std = {testX.std()}")
        testGroups = testData.iloc[:,-1].values    
        testTimestamps = testData.iloc[:,0].values        
        testX = getBuildingFeatures(testData, 'test')  
        
        checkPoint = torch.load(modelPaths[fold], map_location=torch.device(device))
        model = modelToFit(testX.shape[1], 3, CFG.FILTERLIST, CFG.STRIDELIST)
        model.to(device);
        model.load_state_dict(checkPoint['model'])

        # set model to Validate mode
        model.eval()
        ## test Dataset and data loaders
        testDataset = wiFiFeaturesDataset_test(testTimestamps, testX, testGroups)
        testDataloader = getDataLoader(testDataset, datasetType= 'test')

        dataLoaderIterator = iter(testDataloader)
        for idx in range(len(testDataloader)):
            try:
                ts, inputs, testGroups = next(dataLoaderIterator)
            except StopIteration:
                dataLoaderIterator = iter(testDataloader)
                ts, inputs, testGroups = next(dataLoaderIterator)

            inputs = inputs.to(device)
            # forward prediction
            with torch.no_grad():    
                y_preds = model(inputs)

            # store predictions and targets to compute metrics later
            test_ts.append(ts)
            test_preds.append(y_preds)
            test_groups.append(testGroups)
        
        test_fold.append([fold] * len(testX))
        ## del testDataloader
        ## torch.cuda.empty_cache()
        ## gc.collect()
        
    # concatenate to get as 1 2d array 
    test_preds = torch.cat(test_preds, 0).cpu().data.numpy() 
    test_ts = np.concatenate(test_ts, axis=0)
    test_fold = np.concatenate(test_fold, axis=0)
    test_groups = np.concatenate(test_groups, axis=0)
    subm_wifi_df = pd.DataFrame({'timestamp' : test_ts, 'x_preds': test_preds[:,0], 'y_preds': test_preds[:,1],
                                 'floor_preds': test_preds[:,2], 'path' : test_groups, 'fold' : test_fold})
    subm_wifi_df.to_pickle(f"{modelSaveDir}/{buildingName}_wifi_subm.pickle")  
    print(f"Test data prediction for {buildingName} site generated")

## Compute Device as CPU or GPU

In [ ]:
## Device as cpu or tpu
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device('cpu')
print(device)

## Preprocessing classes

In [ ]:
# for cv
folds = GroupKFold(n_splits=CFG.N_FOLDS)

# scaler to handle AMP
## scaler = GradScaler()   

criterion = competitionMetric
modelToFit = CNNWiFiFeaturesModel   ## CNNWiFiFeaturesModel_variant2

## Training & Validation main function

In [ ]:
%%time
if CFG.TRAIN == True:
    
    buildingPathList_train = sorted(glob.glob(f"{wifiFeaturesDir_train}/*.pickle"))
    buildingPathList_train = buildingPathList_train[CFG.BUILDING_SITES_RANGE[0]: CFG.BUILDING_SITES_RANGE[1]]
    print(f"{len(buildingPathList_train)} sites are to be trained")
    ## print(buildingPathList_train)

    for buildingPath_train in buildingPathList_train:
        
        print('----------------------------------')
        ## get building name
        buildingName = getBuildingName(buildingPath_train)
        
        ## train and validate for building data
        ## buildingTrainResults = trainValidateOneBuilding(buildingPath_train, modelToFit)
        ## bestResults = getFoldBestResultsDf(buildingTrainResults)
        
    
        ## generate OOF prediction for building-model combination
        buildingOOF = generateOOF(modelOutputDir, buildingName, CFG.MODEL_NAME)
        
        ## prediction for test data too
        generateWiFiSubmission(modelToFit, modelOutputDir, buildingName, CFG.MODEL_NAME)

        ## save results to file
        buildingOOF.to_pickle(f"{modelOutputDir}/{buildingName}_{CFG.MODEL_NAME}_OOF.pickle")
        
        ## bestResults.to_pickle(f"{modelOutputDir}/{buildingName}_{CFG.MODEL_NAME}_bestResults.pickle")
        ## buildingTrainResults.to_pickle(f"{modelOutputDir}/{buildingName}_{CFG.MODEL_NAME}_trainResults.pickle")
        
        ## plot building results
        ## plotTrainingResults(buildingTrainResults, buildingName)

In [ ]:
## bestResults

```python
buildingPathList_train = sorted(glob.glob(f"{wifiFeaturesDir_train}/*.pickle"))
buildingPathList_test = sorted(glob.glob(f"{wifiFeaturesDir_test}/*.pickle"))

for idx in range(len(buildingPathList_train)):
    print('-----------------------------')
    trainFilePath = buildingPathList_train[idx]
    testFilePath  = buildingPathList_test[idx]
    print(f"{idx}. {getBuildingName(trainFilePath)}")
    with open(trainFilePath, 'rb') as inputTrainFile:
        trainData = pickle.load(inputTrainFile)
    with open(testFilePath, 'rb') as inputTestFile:
        testData = pickle.load(inputTestFile)
    
    with pd.option_context('display.max_rows', 1, 'display.max_columns', 12,
                           'display.width', 500, 'display.precision', 3,
                           'display.colheader_justify', 'left'):
        display(trainData)
        display(testData)
```        